# Walk the Talk (Monitor ESG)
v. 0.1 - Foco em Carbono

## Bibliotecas

In [ ]:
# Instala as bibliotecas necessárias
! pip install PyPDF2
! pip install spacy

In [1]:
# Importa as bibliotecas e módulos necessários
import PyPDF2
import re
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
import spacy
spacy.cli.download("pt_core_news_lg")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


## Funções

### THE TALK
Raspagem de fontes internas (relatórios ESG)

#### Análise do relatório

In [91]:
def analisa_relatorio(empresa, url, termos):

  # Etapa 1: requisita o PDF e salva em uma variável

  # Truque para evitar que o servidor bloqueie o acesso
  headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'
  }

  response = requests.get(caminho_pdf, headers=headers)

  if response.status_code != 200:
    print(f'Erro! A requisição à URL retornou o status {response.status_code}')

  else:

    # Salva o arquivo baixado
    with open(f'{empresa}.pdf', 'wb') as pdf_file:
        pdf_file.write(response.content)
        print(f"O relatório da empresa {empresa} foi baixado com sucesso.")

    # Reabre o arquivo no modo de leitura binária para uso com PyPDF2
    with open(f'{empresa}.pdf', 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

  # Etapa 2: conversão do PDF em string + limpeza do texto

        pdf_to_string = ''
        for x in range(0, len(pdf_reader.pages)):
            pagina = pdf_reader.pages[x]
            texto_pagina = pagina.extract_text()
            pdf_to_string = pdf_to_string + texto_pagina
        print(f"O relatório foi convertido em string.")

    # Lista de substituições a serem feitas
    substituicoes = [
    (r'-\n', ''),  # Substitui hífen seguido por quebra de linha
    (r'\n', ' ') # Substitui todas as quebras de linha por espaço
    # Adicionar outras se necessário
    ]

    # Realiza substituições
    for padrao, substituicao in substituicoes:
        texto = re.sub(padrao, substituicao, pdf_to_string)

    # Remove espaços em branco extra
    texto = ' '.join(texto.split())

    # Etapa 3: processamento do texto com NLP para obtenção de frases

    # Carrega o modelo NLP em português
    nlp = spacy.load("pt_core_news_lg")
    doc = nlp(texto)

    # Filtra frases que contêm pelo menos um dos termos de interesse
    frases = [sent.text for sent in doc.sents if any(termo in sent.text for termo in termos)]

    # Etapa 4: criação e inserção de dados no dataframe

    df = pd.DataFrame(columns=['empresa', 'frase'])
    for frase in frases:
      df.loc[len(df.index)] = [empresa, frase]

    print(f'Foram selecionadas {len(df)} frases do relatório da empresa {empresa}.')

    return df

### THE WALK
Raspagem de fontes externas:
- Bloomberg Línea Brasil (API oculta)
- Folha de S. Paulo (HTML)

#### Raspagem Bloomberg Línea
- O robô acessa as notícias mais recentes do canal ESG do portal e procura pelo nome da empresa e termos de interesse indicados
- O número de registros pode ser indicado como parâmetro na função

In [2]:
# ROBÔ BLOOMBERG LINEA (ARTIGOS)

# Os parâmetros passados são o nome da empresa a ser pesquisada (string), os termos de interesse (lista) e o número de artigos a serem raspados (int)
# A função retorna um dataframe com 5 colunas: data do artigo, título do artigo, nome do veículo, url e conteúdo dos artigos contendo o nome da empresa e os termos de interesse

def raspa_bloomberg(empresa, termos, num_artigos):

  # Cria um dataframe Pandas com 5 colunas
  df = pd.DataFrame(columns=['Titulo', 'Veiculo', 'URL', 'Artigo', 'Data_Publicacao'])

  nome_veiculo = 'Bloomberg Línea'

  # ETAPA 1: Na página do tópico ESG, encontrar os links para as matérias mais recentes de acordo com num_artigos

  # cabeçalhos da busca pela API oculta
  headers = {
    'authority': 'www.bloomberglinea.com.br',
    'accept': '*/*',
    'accept-language': 'pt,en-US;q=0.9,en;q=0.8,es;q=0.7',
    'cache-control': 'no-cache',
    'dnt': '1',
    'if-modified-since': '1700071058771',
    'pragma': 'no-cache',
    'referer': 'https://www.bloomberglinea.com.br/esg/',
    'sec-ch-ua': '"Microsoft Edge";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
  }

  # parâmetros da busca pela API oculta
  # o valor de 'feedSize' pode ser customizado para num_artigos
  params = {
    'query': '{"excludeSections":"/videos","feature":"T1-Small-Small-Small-Small","feedOffset":0,"feedSize":' + str(num_artigos) + ',"includeSections":"/esg"}',
    'filter': '{content_elements{description{basic},display_date,headlines{basic},last_updated_date,taxonomy{primary_section{name,path}},websites{bloomberg-linea-brasil{website_url}}}}',
    'd': '1525',
    '_website': 'bloomberg-linea-brasil',
  }

  # requisita o endpoint da API
  response = requests.get(
      'https://www.bloomberglinea.com.br/pf/api/v3/content/fetch/story-feed-sections',
      params=params,
      headers=headers,
  )

  # Verificar se a requisição foi bem-sucedida (código de status 200)
  if response.status_code == 200:

    print('Bloomberg Línea visitado com sucesso.')

    # interpreta como json
    bloomberg_esg = response.json()['content_elements']

    # cria uma lista para receber as URLs de cada matéria dentro do tópico
    links_bloomberg = []

    # adiciona cada URL encontrada à lista
    for item in bloomberg_esg:

      # Acessar o valor de 'website_url'
      website_url = item.get('websites', {}).get('bloomberg-linea-brasil', {}).get('website_url')

      # Adiciona o valor de 'website_url' à lista
      links_bloomberg.append(f'https://bloomberglinea.com.br{website_url}')

  else:
    print(f'ERRO. A requisição a Bloomberg Linea retornou o status {response.status_code}.')

  # ETAPA 2: Visita cada link encontrado e procura pelo nome da empresa

  for url in links_bloomberg:

    # Requisita a página via requests
    response = requests.get(url)

    # Verifica se a requisição foi bem-sucedida (código de status 200)
    if response.status_code == 200:

      # Informa o status da requisição
      print(f'{url} OK.')

      # Cria um objeto BeautifulSoup com o conteúdo HTML da página
      soup = BeautifulSoup(response.text, 'html.parser')

      # Encontra o título principal
      titulo = soup.find('h1').text

      # Encontra a data de publicação
      data = soup.find('div', {'class' : 'mt-2 mb-0 lg:mb-1'}).find('small').text

      # Encontra todos os parágrafos
      paragrafos = soup.find_all('p')

      # Verificar se o nome da empresa está presente em pelo menos um parágrafo
      empresa_encontrada = any(empresa in paragrafo.get_text() for paragrafo in paragrafos)

      # Se o nome da empresa encontrado, adiciona ao dataframe os parágrafos que contêm pelo menos um dos termos de interesse
      if empresa_encontrada:
        print(f'O nome da empresa {empresa} foi encontrado neste texto.')

        # Verifica se pelo menos um dos termos de interesse está presente em algum parágrafo
        termo_encontrado = any(any(termo.lower() in paragrafo.get_text().lower() for termo in termos) for paragrafo in paragrafos)

        # Se o termo for encontrado, salva no dataframe
        if termo_encontrado:
          print(f'Um termo de interesse foi encontrado neste texto.')
          txt = ''
          for paragrafo in paragrafos:
            txt = txt + paragrafo.text
          df.loc[len(df.index)] = [titulo, nome_veiculo, url, txt, data]
        else: print(f'Nenhum dos termos de interesse foi encontrado neste texto.')
      else:
        print(f'O nome da empresa {empresa} não foi encontrado neste texto.')
    else:
      print(f'ERRO. A requisição a {url} retornou o status {response.status_code}.')

    time.sleep(2) # dorme por 2 segundos

  return df


#### Raspagem Folha
- O robô realiza uma busca pelo nome da empresa e termos de interesse nas notícias do último ano

In [98]:
def raspa_folha(empresa, termos):

    # Cria um dataframe Pandas com 5 colunas
    df = pd.DataFrame(columns=['Titulo', 'Veiculo', 'URL', 'Artigo', 'Data_Publicacao'])

    nome_veiculo = 'Folha de S. Paulo'
    links_folha = []

    # ETAPA 1: Busca pelas matérias que contenham tanto o nome da empresa quanto os termos de interesse

    for termo in termos:

        # Monta a URL de busca a partir dos parâmetros
        # Substitua todos os espaços em branco por '+'
        busca = termo.replace(" ", "+")

        url = f'https://search.folha.uol.com.br/search?q={empresa}+{busca}&periodo=ano&sd=&ed=&site=todos'
        print(url)

        # requisita a primeira url de busca
        response = requests.get(url)

        if response.status_code == 200:
            print(f'{nome_veiculo} visitado com sucesso.')
            sopa = BeautifulSoup(response.text, 'html.parser')
            resultados = sopa.find('div', {'class' : 'c-search__result'}).get_text()
            
            # Encontre todos os números na string
            num_resultados = re.findall('\d+', resultados)

            # O resultado é uma lista de strings, então precisamos converter o primeiro elemento para int
            if len(num_resultados) > 0:
                num_resultados = int(num_resultados[0])

                # Se houver resultados, calcula o número de páginas de resultados (cada página tem 25 resultados)
                num_paginas = num_resultados // 25 + 1
                print(f'A busca por {empresa} e {termo} retornou {num_resultados} artigos em {num_paginas} páginas.')

                # Cria uma lista para receber as URLs de cada matéria na primeira página de busca (ou única página, se for o caso)
                links = sopa.find_all('div', {'class' : 'c-headline__content'})
                links = [link.find('a').get('href') for link in links if '#' not in link.find('a').get('href')]
                print(f'Página 1 de {num_paginas} visitada, com {len(links)} links adicionados.')

                # Se houver apenas uma página de resultados, adiciona os links à lista
                if num_paginas == 1:
                    links_folha = links_folha + links            

                # Se houver mais de uma página de resultados, adiciona os links à lista e visita as páginas restantes
                else:
                    links_folha = links_folha + links
                    pagina = 0
                    while pagina < (num_paginas -1):
                        pagina = pagina + 1
                        print(f'Página {pagina + 1} de {num_paginas} visitada.')
                        url = f'https://search.folha.uol.com.br/search?q={empresa}+{busca}&periodo=ano&sd=&ed=&site=todos&results_count={num_resultados}&sr={pagina * 25 + 1}'
                        print(url)
                        response = requests.get(url)
                        sopa = BeautifulSoup(response.text, 'html.parser')
                        links = []
                        links = sopa.find_all('div', {'class' : 'c-headline__content'})
                        links_folha.extend([link.find('a').get('href') for link in links if link.find('a').get('href') not in links_folha and '#' not in link.find('a').get('href')])
            else:
                print(f'A busca por {empresa} e {termo} não retornou resultados.')

    len(links_folha)

    # ETAPA 2: Visita cada link encontrado e salva o conteúdo dos artigos que contêm o nome da empresa e os termos de interesse

    df = pd.DataFrame(columns=['Titulo', 'Veiculo', 'URL', 'Artigo', 'Data_Publicacao'])

    for link in links_folha:
        
            # Requisita a página via requests
            response = requests.get(link)
        
            # Verifica se a requisição foi bem-sucedida (código de status 200)
            if response.status_code == 200:
        
                # Informa o status da requisição
                print(f'{link} OK.')
        
                # Decodifica o conteúdo da resposta usando 'utf-8'
                content = response.content #.decode('utf-8')

                # Cria um objeto BeautifulSoup com o conteúdo HTML da página
                soup = BeautifulSoup(content, 'html.parser')

                # Encontra o título principal
                titulo = soup.find('h1', class_='c-content-head__title').get_text().strip()
        
                # Encontra a data de publicação

                # Find the 'time' element with the specified class
                time_element = soup.find('time', class_='c-more-options__published-date')

                # Get the value of 'datetime' attribute
                data = time_element['datetime']
                    
                # Encontra todos os parágrafos
                paragrafos = soup.find('div', class_='c-news__body').find_all('p')

                # Extrai o texto de cada parágrafo e junta tudo em uma única string
                texto = ' '.join(paragrafo.get_text().strip() for paragrafo in paragrafos) 

                df.loc[len(df.index)] = [titulo, nome_veiculo, link, texto, data]
                
            else:
                print(f'ERRO. A requisição a {link} retornou o status {response.status_code}.')
        
            time.sleep(3) # dorme por 3 segundos
    
    return df

## Aperta o play

- Os termos de interesse podem ser adicionados à lista conforme desejado
- URLs de relatórios ESG já testadas:
    - Petrobras: https://sustentabilidade.petrobras.com.br/documents/1449993/80c5cf69-cb78-5ae4-aa27-46f958da64ba 

#### Variáveis

In [92]:
# Lista com os termos de interesse
termos = ['carbono', 'co2', 'descarbonização', 'net zero']

# Caminho do PDF a ser analisado
caminho_pdf = 'https://sustentabilidade.petrobras.com.br/documents/1449993/80c5cf69-cb78-5ae4-aa27-46f958da64ba'

# Nome da empresa a ser pesquisada no noticiário (em minúsculas)
empresa = 'Petrobras'

### Execução

In [ ]:
the_talk = analisa_relatorio(empresa, caminho_pdf, termos)

the_walk = pd.concat([raspa_bloomberg(empresa, termos, 10), raspa_folha(empresa, termos)])

In [96]:
the_walk

,Titulo,Veiculo,URL,Artigo,Data_Publicacao
0,Líderes vão à COP28 sob calor recorde e com em...,Bloomberg Línea,https://bloomberglinea.com.br/esg/lideres-vao-...,Expectativa é de que encontro da ONU resulte e...,"29 de Novembro, 2023 | 03:37 PM"
0,"Na COP28, Brasil e mais 115 países preveem tri...",Folha de S. Paulo,https://www1.folha.uol.com.br/ambiente/2023/12...,"O presidente da COP28, conferência do clima da...",2023-12-02 19:25:00
1,Petrobras aprova retomada de obras de refinari...,Folha de S. Paulo,https://www1.folha.uol.com.br/mercado/2023/11/...,A Petrobras aprovou em seu novo plano estratég...,2023-11-27 20:55:00
2,Agenda verde entra no radar da Câmara às véspe...,Folha de S. Paulo,https://www1.folha.uol.com.br/mercado/2023/11/...,"O presidente da Câmara dos Deputados, Arthur L...",2023-11-27 08:00:00
3,"Quente, fervendo",Folha de S. Paulo,https://www1.folha.uol.com.br/opiniao/2023/11/...,"No Sul, chuvas torrenciais assolam cidades e l...",2023-11-25 22:00:00
...,...,...,...,...,...
59,"Mundo não deixará de precisar de petróleo, diz...",Folha de S. Paulo,https://www1.folha.uol.com.br/ambiente/2023/05...,Primeiro diretor da Petrobras voltado para tra...,2023-05-31 11:00:00
60,"De carro popular à Foz do Amazonas, projetos c...",Folha de S. Paulo,https://www1.folha.uol.com.br/ambiente/2023/05...,A discussão em torno da exploração de petróleo...,2023-05-29 19:51:00
61,Térmicas a carvão no Sul pesam nas emissões de...,Folha de S. Paulo,https://www1.folha.uol.com.br/ambiente/2023/10...,As térmicas a carvão foram os destaques em emi...,2023-10-19 12:00:00
62,Semântica petroleira,Folha de S. Paulo,https://www1.folha.uol.com.br/opiniao/2023/03/...,Imagine se em fevereiro do ano passado Vladimi...,2023-03-09 21:00:00


In [ ]:
petrobras_talk

In [42]:
# Lista com os termos de interesse
termos = ['carbono', 'co2', 'descarbonização', 'net zero']

# Caminho do PDF a ser analisado
caminho_pdf = 'https://api.mziq.com/mzfilemanager/v2/d/80f2e993-0a30-421a-9470-a4d5c8ad5e9f/b411dff3-201d-68fa-29bb-42b49a250f19?origin=2'

# Nome da empresa a ser pesquisada no noticiário (em minúsculas)
empresa = 'Itaú'

In [ ]:
bradesco_walk = raspa_bloomberg(empresa, termos, 15)

#bradesco_talk = analisa_relatorio(empresa, caminho_pdf, termos)

In [29]:
bradesco_walk

,Titulo,Veiculo,URL,Artigo,Data_Publicacao


In [81]:
def raspa_folha(empresa, termos):

    # Cria um dataframe Pandas com 5 colunas
    df = pd.DataFrame(columns=['Titulo', 'Veiculo', 'URL', 'Artigo', 'Data_Publicacao'])

    nome_veiculo = 'Folha de S. Paulo'
    links_folha = []

    # ETAPA 1: Busca pelas matérias que contenham tanto o nome da empresa quanto os termos de interesse

    for termo in termos:

        # Monta a URL de busca a partir dos parâmetros
        # Substitua todos os espaços em branco por '+'
        busca = termo.replace(" ", "+")

        url = f'https://search.folha.uol.com.br/search?q={empresa}+{busca}&periodo=ano&sd=&ed=&site=todos'
        print(url)

        # requisita a primeira url de busca
        response = requests.get(url)

        if response.status_code == 200:
            print(f'{nome_veiculo} visitado com sucesso.')
            sopa = BeautifulSoup(response.text, 'html.parser')
            resultados = sopa.find('div', {'class' : 'c-search__result'}).get_text()
            
            # Encontre todos os números na string
            num_resultados = re.findall('\d+', resultados)

            # O resultado é uma lista de strings, então precisamos converter o primeiro elemento para int
            if len(num_resultados) > 0:
                num_resultados = int(num_resultados[0])
        
                # Se houver resultados, calcula o número de páginas de resultados (cada página tem 25 resultados)
                num_paginas = num_resultados // 25 + 1
                print(f'A busca por {empresa} e {termo} retornou {num_resultados} artigos em {num_paginas} páginas.')

                # Cria uma lista para receber as URLs de cada matéria na primeira página de busca (ou única página, se for o caso)
                links = sopa.find_all('div', {'class' : 'c-headline__content'})
                links = [link.find('a').get('href') for link in links if '#' not in link.find('a').get('href')]
                print(f'Página 1 de {num_paginas} visitada, com {len(links)} links adicionados.')

                # Se houver apenas uma página de resultados, adiciona os links à lista
                if num_paginas == 1:
                    links_folha = links_folha + links            

                # Se houver mais de uma página de resultados, adiciona os links à lista e visita as páginas restantes
                else:
                    links_folha = links_folha + links
                    pagina = 0
                    while pagina < (num_paginas -1):
                        pagina = pagina + 1
                        print(f'Página {pagina + 1} de {num_paginas} visitada.')
                        url = f'https://search.folha.uol.com.br/search?q={empresa}+{busca}&periodo=ano&sd=&ed=&site=todos&results_count={num_resultados}&sr={pagina * 25 + 1}'
                        print(url)
                        response = requests.get(url)
                        sopa = BeautifulSoup(response.text, 'html.parser')
                        links = []
                        links = sopa.find_all('div', {'class' : 'c-headline__content'})
                        links_folha.extend([link.find('a').get('href') for link in links if link.find('a').get('href') not in links_folha and '#' not in link.find('a').get('href')])
            else:
                print(f'A busca por {empresa} e {termo} não retornou resultados.')

    len(links_folha)

    # ETAPA 2: Visita cada link encontrado e salva o conteúdo dos artigos que contêm o nome da empresa e os termos de interesse

    df = pd.DataFrame(columns=['Titulo', 'Veiculo', 'URL', 'Artigo', 'Data_Publicacao'])

    for link in links_folha:
        
            # Requisita a página via requests
            response = requests.get(link)
        
            # Verifica se a requisição foi bem-sucedida (código de status 200)
            if response.status_code == 200:
        
                # Informa o status da requisição
                print(f'{link} OK.')
        
                # Decodifica o conteúdo da resposta usando 'utf-8'
                content = response.content.decode('utf-8')

                # Cria um objeto BeautifulSoup com o conteúdo HTML da página
                soup = BeautifulSoup(content, 'html.parser')

                # Encontra o título principal
                titulo = soup.find('h1', class_='c-content-head__title').get_text().strip()
        
                # Encontra a data de publicação

                # Find the 'time' element with the specified class
                time_element = soup.find('time', class_='c-more-options__published-date')

                # Get the value of 'datetime' attribute
                data = time_element['datetime']
                    
                # Encontra todos os parágrafos
                paragrafos = soup.find('div', class_='c-news__body').find_all('p')

                # Extrai o texto de cada parágrafo e junta tudo em uma única string
                texto = ' '.join(paragrafo.get_text().strip() for paragrafo in paragrafos) 

                df.loc[len(df.index)] = [titulo, nome_veiculo, link, texto, data]
                
            else:
                print(f'ERRO. A requisição a {link} retornou o status {response.status_code}.')
        
            time.sleep(3) # dorme por 3 segundos
    
    return df


In [83]:
empresa = 'Bradesco'
termos = ['carbono', 'co2', 'descarbonização', 'net zero']

bbdc_walk = pd.concat([raspa_bloomberg(empresa, termos, 15), raspa_folha(empresa, termos)])

Bloomberg Línea visitado com sucesso.
https://bloomberglinea.com.br/esg/relatorio-da-onu-aponta-qual-e-a-ameaca-silenciosa-do-aquecimento-global/ OK.
O nome da empresa Bradesco não foi encontrado neste texto.
https://bloomberglinea.com.br/esg/lideres-vao-a-cop28-sob-calor-recorde-e-com-emissoes-de-carbono-dos-paises-em-alta/ OK.
O nome da empresa Bradesco não foi encontrado neste texto.
https://bloomberglinea.com.br/2023/09/19/cop28-o-que-e-a-conferencia-do-clima-e-por-que-ela-e-importante/ OK.
O nome da empresa Bradesco não foi encontrado neste texto.
https://bloomberglinea.com.br/esg/brasil-chega-a-cop28-com-delegacao-de-15-ministros-e-aposta-alta-na-lideranca-climatica/ OK.
O nome da empresa Bradesco não foi encontrado neste texto.
https://bloomberglinea.com.br/2023/11/28/blackrock-reforma-em-bancos-de-desenvolvimento-liberaria-us4-tri-para-transicao-verde/ OK.
O nome da empresa Bradesco não foi encontrado neste texto.
https://bloomberglinea.com.br/agro/vibra-expansao-do-milho-abre-